In [ ]:
import clip
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import utils_cub as ut
import numpy as np

In [ ]:
# set parameters

root  = "/path/to/repository/SpeciesRecognition/"

model_architecture = 'ViT-B/32' #'RN50'

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(model_architecture, device)

batch_size = 32 
validation_split  = 0.1

In [ ]:
def get_features(dataloader):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

def save_features(name, data):
    np.save(name, data)

def load_features(name):
    return np.load(name)

In [ ]:
# get features

train_data_loader, val_data_loader, test_data_loader  = ut.get_CUB_dataloader(root, batch_size, validation_split, preprocess)

train_features, train_labels = get_features(train_data_loader)
val_features, val_labels = get_features(val_data_loader)
test_features, test_labels = get_features(test_data_loader)

In [ ]:
#save features 

nn = model_architecture.replace(
        "/", "-").lower()

ut.save_features('train_features_cub_' + nn + '.npy', train_features)
ut.save_features('train_labels_cub_' + nn + '.npy', train_labels)
ut.save_features('val_features_cub_' + nn + '.npy', val_features)
ut.save_features('val_labels_cub_' + nn + '.npy', val_labels)
ut.save_features('test_features_cub_' + nn + '.npy', test_features)
ut.save_features('test_labels_cub_' + nn + '.npy', test_labels)
